In [1]:
import folium
import pandas as pd
import geopandas as gpd
import glob
import re
import numpy as np
from tqdm.auto import tqdm

In [259]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.reset_option('max_rows')
# pd.reset_option('max_columns')

In [3]:
geo_countries = gpd.read_file('../data/countries.geojson')
geo_countries.head()

,ADMIN,ISO_A3,geometry
0,Aruba,ABW,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Afghanistan,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."
2,Angola,AGO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,AIA,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,ALB,"POLYGON ((19.74777 42.57890, 19.74601 42.57993..."


In [4]:
world_cities = pd.read_csv('../data/worldcities/worldcities.csv', usecols= ['city_ascii', 'country', 'iso3', 'population'])
world_cities.head()

,city_ascii,country,iso3,population
0,Tokyo,Japan,JPN,39105000.0
1,Jakarta,Indonesia,IDN,35362000.0
2,Delhi,India,IND,31870000.0
3,Manila,Philippines,PHL,23971000.0
4,Sao Paulo,Brazil,BRA,22495000.0


In [11]:
city_iso3 = world_cities.sort_values('population', ascending = False).drop_duplicates('city_ascii').rename(columns={'city_ascii':'city'})[['city', 'iso3']]
state_iso3 = pd.read_csv('../data/US_states.csv', usecols=['STATE', 'STATE2']).assign(iso3='USA').rename(columns = {'STATE':'state', 'STATE2':'state2'})
country_iso3 = world_cities.sort_values('population', ascending = False).drop_duplicates('iso3')[['country', 'iso3']]

In [12]:
country_iso3.head()

,country,iso3
0,Japan,JPN
1,Indonesia,IDN
2,India,IND
3,Philippines,PHL
4,Brazil,BRA


In [ ]:
country_pop = (world_cities.groupby('iso3').
 sum('population').
 drop(columns = ['lat', 'lng', 'id']).
 rename(columns = {'population':'country_pop'}).
 reset_index()
)
country_pop = country_pop[country_pop['iso3'] != 'CHN']


In [ ]:
country_pop.sort_values('country_pop', ascending = False)

In [ ]:
# initialize the map and store it in a m object
m = folium.Map(location=[40, -95], zoom_start=4)

folium.Choropleth(
    geo_data='../data/countries.geojson',
    name="choropleth",
    data=country_pop,
    columns=["iso3", "country_pop"],
    key_on="feature.properties.ISO_A3",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=.1,
    legend_name="population",
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
df.value_counts('location', ascending = False).head(100)

In [414]:
df.query('location == "Italia"')['iso_final'][:5]

7       NaN
1568    NaN
3582    NaN
6598    NaN
6658    NaN
Name: iso_final, dtype: object

In [402]:
country_iso3[country_iso3['country'].str.contains('Czech')]

,country,iso3
519,Czechia,CZE


### Convet location to iso3

In [253]:
# Function to replace the inconsistent locations with country names
US = ['US', 'USA', 'Southern California', 'United States of America', 'West Coast, USA (SEA/GEG/LAX)',
       'Democrat in the USA','Washington DC', 'New York City', 'NYC', 'USA (SEA/GEG/LAX)', 'Washington DC',
       'Lost in the Minnesota North Woods', 'Washington D.C.', 'U.S.A.', 'U.S.A', 'us', 'New England', 'Philly','D.C.'
       ,'usa', 'Northern Virginia', 'American', 'South Florida','Pacific Northwest', 'Northern California', 
       'San Diego & New Orleans', 'San Francisco Bay Area','New York and the World', 'America','Washington State', 
       'Midwest','East Coast', 'US of A', 'Harrisburg Pa']
UK = ['UK','England', 'Scotland', 'london', 'Wales', 'uk', 'West Saxons', 'LONDON', 'South London', 'Northern Ireland',
       'Somerset England UK Eu', 'Derbyshire', 'England and International', 'London UK','Leicestershire','Lancashire'
       ,'West Sussex','North Yorkshire','Yorkshire and The Humber, Engl','Yorkshire and The Humber','Newcastle upon Tyne'
       ,'Staffordshire','Oxfordshire','Cardiff','West Yorkshire','Cymru','Great Britain','North Wales','Wiltshire'
       ,'Stockport','North West England','Cambridgeshire','glasgow','Romford','SCOTLAND','South Wales','Dorset'
       ,'some were in UK','england','Hertfordshire','Shropshire England','North East England','Hampshire UK'
       ,'Norfolk England','Dorset, England.','Bristol UK','Republic of Wales','U.K.','Lancashire','East Sussex'
       ,'Warwickshire', 'Edinburgh ~ Heart of Scotland!']
Germany = ["Deutschland", 'Somewhere in Germany']
Ukraine = ['Ucraina', 'chernivtsi', 'Some Future Place in Ukraine', 'Mariouple']
Poland = ['Polska']
Canada = ['CANADA']
Netherlands = ['The Netherlands']
Czechia = ['Czech Republic']
Italy = ['Italia']    
Brazil = ['Brasil', 'Brazil Brazil']   
Finland = ['Suomi', "East-Finnish People's Republic"]    
Japan = ['Okinotorishima Ogasawara Tokyo']   
Australia = ['Queensland','Australia - International']   
Sweden = ['Sverige']  
Kazakhstan = ['Astana']  
Serbia = ['Belgrade City'] 
India = ['mumbai']   
Belgium = ['Belgique', 'Bruxelles']  
Ethiopia = ['Tigray']   
Venezuela = ['Venezuela revolucionaria']
Spain = ['Tarragona']
Greece = ['Athens Greece']
Denmark = ['Danmark']

def convert_loc(text):
    if text in US:
        return 'United States'
    if text in UK:
        return 'United Kingdom'
    if text in Germany:
        return 'Germany'
    if text in Ukraine:
        return 'Ukraine'
    if text in Poland:
        return 'Poland'
    if text in Canada:
        return 'Canada'
    if text in Netherlands:
        return 'Netherlands'
    if text in Czechia:
        return 'Czechia'
    if text in Italy:
        return 'Italy'
    if text in Brazil:
        return 'Brazil'
    if text in Finland:
        return 'Finland'
    if text in Japan:
        return 'Japan'
    if text in Australia:
        return 'Australia'
    if text in Sweden:
        return 'Sweden'
    if text in Kazakhstan:
        return 'Kazakhstan'
    if text in Serbia:
        return 'Serbia'
    if text in India:
        return 'India'
    if text in Belgium:
        return 'Belgium'
    if text in Ethiopia:
        return 'Ethiopia'
    if text in Venezuela:
        return 'Venezuela'
    if text in Spain:
        return 'Spain'
    if text in Greece:
        return 'Greece'
    if text in Denmark:
        return 'Denmark'
    else:
        return text 

In [257]:
col_to_keep = ['tweet_id', 'acct_desc', 'date', 'location', 'friends_ount','followers_ount', 'text', 'compound', 'iso3_final']
for filepath in tqdm(glob.glob('../data/tweets_en/*')):
    df = pd.read_csv(filepath, lineterminator='\n', encoding='latin-1')
    
    # split the location by comma and assign them to loc1 and loc2 
    loc_split = df['location'].apply(lambda x : str(x).split(','))
    loc1 = []
    loc2 = []
    for loc in loc_split:
        if len(loc) == 1:
            loc1 += [loc[0]]
            loc2 += ['nan']
        if len(loc) == 2:
            loc1 += [loc[0]]
            loc2 += [loc[1]]
        if len(loc) >2:
            loc1 += [loc[0]]
            loc2 += [loc[1]]
    df = df.assign(loc1 = loc1, loc2 = loc2)
    
    # strip the blank space in loc1 and loc2; convert the inconsistent loc1 and loc2 to country names
    df['loc1'] = df['loc1'].apply(lambda x: x.strip(' '))
    df['loc1'] = df['loc1'].apply(convert_loc)
    df['loc2'] = df['loc2'].apply(lambda x: x.strip(' '))
    df['loc2'] = df['loc2'].apply(convert_loc)
    
    # merge location columns with iso3 dataframes to match city, state, or country
    df = (df.
     merge(city_iso3, how = 'left', left_on = 'loc1', right_on = 'city').
     merge(city_iso3, how = 'left', left_on = 'loc2', right_on = 'city').
     merge(state_iso3, how = 'left', left_on = 'loc1', right_on = 'state').
     merge(state_iso3, how = 'left', left_on = 'loc2', right_on = 'state').
     merge(state_iso3, how = 'left', left_on = 'loc1', right_on = 'state2').
     merge(state_iso3, how = 'left', left_on = 'loc2', right_on = 'state2').
     merge(country_iso3, how = 'left', left_on = 'loc1', right_on = 'country').
     merge(country_iso3, how = 'left', left_on = 'loc2', right_on = 'country')
    )
    
    # ignore the null values and get the iso3
    df['iso3_final'] = (df['iso3_x'].iloc[:,0].
     combine_first(df['iso3_x'].iloc[:,1]).
     combine_first(df['iso3_x'].iloc[:,2]).
     combine_first(df['iso3_x'].iloc[:,3]).
     combine_first(df['iso3_y'].iloc[:,0]).
     combine_first(df['iso3_y'].iloc[:,1]).
     combine_first(df['iso3_y'].iloc[:,2]).
     combine_first(df['iso3_y'].iloc[:,3])
    )
    df = df[col_to_keep]# drop iso3_ columns
    df.to_csv(filepath, index = False)

  0%|          | 0/10 [00:00<?, ?it/s]